# Generic Time Series Forecasting with ML Models (Monthly Data)

This notebook compares multiple machine learning models for **monthly** time series forecasting.

## Supported Models
1. **Linear Regression**
2. **Ridge Regression** (L2 regularization)
3. **Lasso Regression** (L1 regularization)
4. **Random Forest**
5. **Gradient Boosting**
6. **XGBoost**
7. **LightGBM**
8. **Prophet** (Time series specialized)

## Applicable Use Cases
- Monthly sales forecasting
- Monthly demand prediction
- Monthly inventory planning
- Monthly revenue forecasting
- Monthly financial metrics
- Any monthly aggregated time series data

## Key Differences from Daily Version
- Adapted for monthly granularity (no day-of-week or daily features)
- Optimized seasonality detection for monthly patterns
- Test period specified in months instead of days
- Quarterly and annual patterns emphasized

## 1. Install and Import Libraries

In [ ]:
# Install required libraries
!pip install xgboost lightgbm prophet -q

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split, TimeSeriesSplit, cross_val_score
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error, r2_score
from sklearn.preprocessing import StandardScaler

import xgboost as xgb
import lightgbm as lgb
from prophet import Prophet

import warnings
warnings.filterwarnings('ignore')

# Set font to support unicode minus sign
plt.rcParams['axes.unicode_minus'] = False

print("Libraries loaded successfully!")

## 2. Configuration

**IMPORTANT**: Set your column names here before running the rest of the notebook

In [ ]:
# =============================================================================
# CONFIGURATION - Edit these settings to match your data
# =============================================================================

# Column names in your CSV file
DATE_COLUMN = 'date'           # Column containing dates (YYYY-MM-DD or YYYY-MM format)
TARGET_COLUMN = 'MP'           # Column to predict (your target variable)

# Optional: Columns to exclude from features (besides date and target)
# Example: ['id', 'notes', 'category'] - these won't be used as features
EXCLUDE_COLUMNS = []           # Add column names you want to exclude

# Test period settings
TEST_MONTHS = 6                # Number of months to use for testing (default: 6 months)

# Display settings
TARGET_DISPLAY_NAME = 'Target' # How to display target in graphs
TARGET_SCALE = 1_000_000       # Divide values by this for display (1 for no scaling, 1000 for thousands, etc.)
TARGET_UNIT = 'Million'        # Unit for display (e.g., 'Million', 'Thousand', '')

# =============================================================================

print("Configuration set!")
print(f"  Date column: {DATE_COLUMN}")
print(f"  Target column: {TARGET_COLUMN}")
print(f"  Test period: {TEST_MONTHS} months")

## 3. Load Data

In [ ]:
# Upload file
from google.colab import files
uploaded = files.upload()

In [ ]:
# Load data
if len(uploaded) == 0:
    raise ValueError("Error: No file uploaded. Please run the cell above and upload a CSV file.")

filename = list(uploaded.keys())[0]
df = pd.read_csv(filename, encoding='utf-8')

print(f"Data shape: {df.shape}")
print(f"\nColumn names:")
print(df.columns.tolist())
print(f"\nFirst 5 rows:")
df.head()

## 4. Data Preprocessing

In [ ]:
# Clean column names (remove spaces)
df.columns = df.columns.str.strip()

# Validate required columns exist
if DATE_COLUMN not in df.columns:
    raise ValueError(f"Error: Date column '{DATE_COLUMN}' not found in data. Available columns: {df.columns.tolist()}")
if TARGET_COLUMN not in df.columns:
    raise ValueError(f"Error: Target column '{TARGET_COLUMN}' not found in data. Available columns: {df.columns.tolist()}")

# Convert date column to datetime and extract year-month
df[DATE_COLUMN] = pd.to_datetime(df[DATE_COLUMN])
# Normalize to first day of month for consistency
df[DATE_COLUMN] = df[DATE_COLUMN].dt.to_period('M').dt.to_timestamp()

# Clean target column (remove commas and quotes if present, convert to numeric)
if df[TARGET_COLUMN].dtype == 'object':
    df[TARGET_COLUMN] = df[TARGET_COLUMN].astype(str).str.replace(',', '').str.replace('"', '').str.strip()
df[TARGET_COLUMN] = pd.to_numeric(df[TARGET_COLUMN], errors='coerce')

print("Data preprocessing complete!")
print(f"\nFirst 5 rows after preprocessing:")
df.head()

In [ ]:
# Split into training data (with target) and prediction data (without target)
df_train_full = df[df[TARGET_COLUMN].notna()].copy()
df_predict = df[df[TARGET_COLUMN].isna()].copy()

print(f"Training data: {len(df_train_full)} months ({df_train_full[DATE_COLUMN].min().date()} to {df_train_full[DATE_COLUMN].max().date()})")
print(f"Prediction data: {len(df_predict)} months", end="")
if len(df_predict) > 0:
    print(f" ({df_predict[DATE_COLUMN].min().date()} to {df_predict[DATE_COLUMN].max().date()})")
else:
    print(" (none)")

# Target statistics
print(f"\n{TARGET_COLUMN} statistics:")
print(f"  Mean: {df_train_full[TARGET_COLUMN].mean():,.2f}")
print(f"  Std: {df_train_full[TARGET_COLUMN].std():,.2f}")
print(f"  Min: {df_train_full[TARGET_COLUMN].min():,.2f}")
print(f"  Max: {df_train_full[TARGET_COLUMN].max():,.2f}")

## 5. Feature Engineering

In [ ]:
# Select feature columns (exclude date, target, and configured exclude columns)
exclude_list = [DATE_COLUMN, TARGET_COLUMN] + EXCLUDE_COLUMNS
feature_cols = [col for col in df.columns if col not in exclude_list]

print(f"Using {len(feature_cols)} feature columns from original data:")
print(feature_cols)

In [ ]:
# Add time-based features optimized for MONTHLY data
def add_time_features(df, date_col):
    """
    Add time-based features for monthly time series data.
    Focuses on yearly, quarterly, and monthly patterns.
    """
    df = df.copy()
    
    # Basic temporal features
    df['year'] = df[date_col].dt.year
    df['month'] = df[date_col].dt.month
    df['quarter'] = df[date_col].dt.quarter
    
    # Relative time (months since start)
    df['months_since_start'] = (df[date_col].dt.year - df[date_col].dt.year.min()) * 12 + \
                                (df[date_col].dt.month - df[date_col].dt.month.iloc[0])
    
    # Cyclical features (sin/cos transformation for seasonality)
    # Monthly seasonality (12-month cycle)
    df['month_sin'] = np.sin(2 * np.pi * df['month'] / 12)
    df['month_cos'] = np.cos(2 * np.pi * df['month'] / 12)
    
    # Quarterly seasonality (4-quarter cycle)
    df['quarter_sin'] = np.sin(2 * np.pi * df['quarter'] / 4)
    df['quarter_cos'] = np.cos(2 * np.pi * df['quarter'] / 4)
    
    # Seasonal indicators
    df['is_q1'] = (df['quarter'] == 1).astype(int)
    df['is_q2'] = (df['quarter'] == 2).astype(int)
    df['is_q3'] = (df['quarter'] == 3).astype(int)
    df['is_q4'] = (df['quarter'] == 4).astype(int)
    
    # Year start/end indicators (often have special patterns)
    df['is_year_start'] = (df['month'] == 1).astype(int)
    df['is_year_end'] = (df['month'] == 12).astype(int)
    
    # Quarter start/end indicators
    df['is_quarter_start'] = df['month'].isin([1, 4, 7, 10]).astype(int)
    df['is_quarter_end'] = df['month'].isin([3, 6, 9, 12]).astype(int)
    
    return df

df_train_full = add_time_features(df_train_full, DATE_COLUMN)
df_predict = add_time_features(df_predict, DATE_COLUMN)

# Extended feature list
time_features = ['year', 'month', 'quarter', 'months_since_start',
                 'month_sin', 'month_cos', 'quarter_sin', 'quarter_cos',
                 'is_q1', 'is_q2', 'is_q3', 'is_q4',
                 'is_year_start', 'is_year_end', 'is_quarter_start', 'is_quarter_end']
feature_cols_extended = feature_cols + time_features

print(f"\nTotal features after engineering: {len(feature_cols_extended)}")
print(f"\nTime-based features added (optimized for monthly data):")
print(time_features)

## 6. Train/Test Split

For time series data, we use the last period as test data

In [ ]:
# Sort by date
df_train_full = df_train_full.sort_values(DATE_COLUMN).reset_index(drop=True)

# Split into train and test
if len(df_train_full) <= TEST_MONTHS:
    raise ValueError(f"Error: Not enough data. Need more than {TEST_MONTHS} months, but only have {len(df_train_full)} months")

train_df = df_train_full.iloc[:-TEST_MONTHS].copy()
test_df = df_train_full.iloc[-TEST_MONTHS:].copy()

print(f"Training data: {len(train_df)} months ({train_df[DATE_COLUMN].min().date()} to {train_df[DATE_COLUMN].max().date()})")
print(f"Test data: {len(test_df)} months ({test_df[DATE_COLUMN].min().date()} to {test_df[DATE_COLUMN].max().date()})")

# Separate features and target
X_train = train_df[feature_cols_extended]
y_train = train_df[TARGET_COLUMN]
X_test = test_df[feature_cols_extended]
y_test = test_df[TARGET_COLUMN]

print(f"\nX_train shape: {X_train.shape}")
print(f"X_test shape: {X_test.shape}")

## 7. Model Training and Evaluation

In [ ]:
# Evaluation function
def evaluate_model(y_true, y_pred, model_name):
    mae = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    mape = mean_absolute_percentage_error(y_true, y_pred) * 100
    r2 = r2_score(y_true, y_pred)
    
    return {
        'Model': model_name,
        'MAE': mae,
        'RMSE': rmse,
        'MAPE(%)': mape,
        'R2': r2
    }

In [ ]:
# Define models
models = {
    '1. Linear Regression': LinearRegression(),
    '2. Ridge Regression': Ridge(alpha=1.0),
    '3. Lasso Regression': Lasso(alpha=1.0),
    '4. Random Forest': RandomForestRegressor(n_estimators=100, max_depth=10, random_state=42, n_jobs=-1),
    '5. Gradient Boosting': GradientBoostingRegressor(n_estimators=100, max_depth=5, random_state=42),
    '6. XGBoost': xgb.XGBRegressor(n_estimators=100, max_depth=5, learning_rate=0.1, random_state=42),
    '7. LightGBM': lgb.LGBMRegressor(n_estimators=100, max_depth=5, learning_rate=0.1, random_state=42, verbose=-1)
}

# Store results
results = []
predictions = {}

print("Starting model training and evaluation...\n")
print("="*70)

for name, model in models.items():
    print(f"\nTraining {name}...")
    
    # Train
    model.fit(X_train, y_train)
    
    # Predict
    y_pred = model.predict(X_test)
    predictions[name] = y_pred
    
    # Evaluate
    result = evaluate_model(y_test, y_pred, name)
    results.append(result)
    
    print(f"  MAPE: {result['MAPE(%)']:.2f}%, R2: {result['R2']:.4f}")

print("\n" + "="*70)
print("All models trained!")

## 8. Prophet Model (Time Series Specialized)

In [ ]:
print("\nTraining 8. Prophet...")

# Prepare data for Prophet
prophet_train = train_df[[DATE_COLUMN, TARGET_COLUMN]].rename(columns={DATE_COLUMN: 'ds', TARGET_COLUMN: 'y'})
prophet_test = test_df[[DATE_COLUMN, TARGET_COLUMN]].rename(columns={DATE_COLUMN: 'ds', TARGET_COLUMN: 'y'})

# Create Prophet model optimized for MONTHLY data
prophet_model = Prophet(
    yearly_seasonality=True,      # Keep yearly patterns
    weekly_seasonality=False,     # Disable weekly (not applicable for monthly data)
    daily_seasonality=False,      # Disable daily (not applicable for monthly data)
    seasonality_mode='multiplicative',
    changepoint_prior_scale=0.05  # Control flexibility of trend changes
)

# Add quarterly seasonality (custom for monthly data)
prophet_model.add_seasonality(name='quarterly', period=91.25, fourier_order=4)

# Train
prophet_model.fit(prophet_train)

# Predict
future = prophet_test[['ds']].copy()
prophet_forecast = prophet_model.predict(future)
y_pred_prophet = prophet_forecast['yhat'].values
predictions['8. Prophet'] = y_pred_prophet

# Evaluate
result_prophet = evaluate_model(y_test, y_pred_prophet, '8. Prophet')
results.append(result_prophet)

print(f"  MAPE: {result_prophet['MAPE(%)']:.2f}%, R2: {result_prophet['R2']:.4f}")

## 9. Results Comparison

In [ ]:
# Convert to DataFrame
results_df = pd.DataFrame(results)

# Sort by MAPE (lower is better)
results_df = results_df.sort_values('MAPE(%)')

# Format for display
results_display = results_df.copy()
results_display['MAE'] = results_display['MAE'].apply(lambda x: f"{x:,.2f}")
results_display['RMSE'] = results_display['RMSE'].apply(lambda x: f"{x:,.2f}")
results_display['MAPE(%)'] = results_display['MAPE(%)'].apply(lambda x: f"{x:.2f}")
results_display['R2'] = results_display['R2'].apply(lambda x: f"{x:.4f}")

print("\n" + "="*80)
print("Model Comparison Results (sorted by MAPE)")
print("="*80)
print("\n* Lower MAPE (Mean Absolute Percentage Error) = Better accuracy\n")
print(results_display.to_string(index=False))

In [ ]:
# Identify best model
best_model_name = results_df.iloc[0]['Model']
best_mape = results_df.iloc[0]['MAPE(%)']

print("\n" + "*"*80)
print(f"Best Model: {best_model_name}")
print(f"MAPE: {best_mape:.2f}%")
print("*"*80)

In [ ]:
# Accuracy comparison graphs
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# MAPE comparison
ax1 = axes[0]
colors = ['gold' if m == best_model_name else 'steelblue' for m in results_df['Model']]
bars = ax1.barh(results_df['Model'], results_df['MAPE(%)'], color=colors)
ax1.set_xlabel('MAPE (%)', fontsize=12)
ax1.set_title('MAPE by Model (Lower is Better)', fontsize=14)
ax1.invert_yaxis()
for bar, val in zip(bars, results_df['MAPE(%)']):
    ax1.text(val + 0.1, bar.get_y() + bar.get_height()/2, f'{val:.2f}%', va='center')

# R2 comparison
ax2 = axes[1]
colors = ['gold' if m == best_model_name else 'steelblue' for m in results_df['Model']]
bars = ax2.barh(results_df['Model'], results_df['R2'], color=colors)
ax2.set_xlabel('R² Score', fontsize=12)
ax2.set_title('R² by Model (Higher is Better)', fontsize=14)
ax2.invert_yaxis()
for bar, val in zip(bars, results_df['R2']):
    ax2.text(val + 0.01, bar.get_y() + bar.get_height()/2, f'{val:.4f}', va='center')

plt.tight_layout()
plt.show()

## 10. Prediction Visualization

In [ ]:
# Visualize predictions from top 3 models
top_3_models = results_df.head(3)['Model'].tolist()

fig, ax = plt.subplots(figsize=(14, 6))

# Actual values
ax.plot(test_df[DATE_COLUMN], y_test / TARGET_SCALE, 'k-', label='Actual', linewidth=2, alpha=0.8, marker='o')

# Predictions from top 3 models
colors = ['red', 'blue', 'green']
markers = ['s', '^', 'D']
for i, model_name in enumerate(top_3_models):
    ax.plot(test_df[DATE_COLUMN], predictions[model_name] / TARGET_SCALE, 
            '--', color=colors[i], label=model_name, linewidth=1.5, alpha=0.7, marker=markers[i])

ax.set_title('Prediction vs Actual in Test Period (Top 3 Models)', fontsize=14)
ax.set_xlabel('Date')
ax.set_ylabel(f'{TARGET_DISPLAY_NAME} ({TARGET_UNIT})')
ax.legend(loc='upper left')
ax.grid(True, alpha=0.3)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
# Detailed display of best model predictions
fig, axes = plt.subplots(2, 1, figsize=(14, 10))

best_pred = predictions[best_model_name]

# Top: Prediction vs Actual
ax1 = axes[0]
ax1.plot(test_df[DATE_COLUMN], y_test / TARGET_SCALE, 'b-', label='Actual', linewidth=2, marker='o')
ax1.plot(test_df[DATE_COLUMN], best_pred / TARGET_SCALE, 'r--', label=f'{best_model_name} Prediction', linewidth=2, marker='s')
ax1.fill_between(test_df[DATE_COLUMN], y_test / TARGET_SCALE, best_pred / TARGET_SCALE, alpha=0.3, color='gray')
ax1.set_title(f'Best Model ({best_model_name}) - Prediction vs Actual', fontsize=14)
ax1.set_ylabel(f'{TARGET_DISPLAY_NAME} ({TARGET_UNIT})')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Bottom: Prediction errors
ax2 = axes[1]
error_pct = (best_pred - y_test) / y_test * 100
colors = ['red' if e > 0 else 'blue' for e in error_pct]
ax2.bar(test_df[DATE_COLUMN], error_pct, color=colors, alpha=0.7)
ax2.axhline(y=0, color='black', linestyle='-', linewidth=0.5)
ax2.axhline(y=error_pct.mean(), color='green', linestyle='--', label=f'Mean Error: {error_pct.mean():.2f}%')
ax2.set_title('Monthly Prediction Error (%)', fontsize=14)
ax2.set_ylabel('Error (%)')
ax2.set_xlabel('Date')
ax2.legend()
ax2.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

## 11. Feature Importance (Tree-based Models)

In [ ]:
# Display feature importance for LightGBM
lgb_model = models['7. LightGBM']

# Get feature importance
importance = pd.DataFrame({
    'Feature': feature_cols_extended,
    'Importance': lgb_model.feature_importances_
}).sort_values('Importance', ascending=False)

# Display top 15 features
fig, ax = plt.subplots(figsize=(10, 8))
top_n = min(15, len(importance))
ax.barh(importance.head(top_n)['Feature'], importance.head(top_n)['Importance'], color='steelblue')
ax.set_xlabel('Importance')
ax.set_title(f'Feature Importance (LightGBM, Top {top_n})', fontsize=14)
ax.invert_yaxis()
plt.tight_layout()
plt.show()

print(f"\nFeature Importance (Top {top_n}):")
print(importance.head(top_n).to_string(index=False))

## 12. Future Prediction

In [ ]:
# Retrain best model on full data
print(f"Retraining best model ({best_model_name}) on full data...")

X_full = df_train_full[feature_cols_extended]
y_full = df_train_full[TARGET_COLUMN]

# Recreate model
if 'LightGBM' in best_model_name:
    final_model = lgb.LGBMRegressor(n_estimators=100, max_depth=5, learning_rate=0.1, random_state=42, verbose=-1)
elif 'XGBoost' in best_model_name:
    final_model = xgb.XGBRegressor(n_estimators=100, max_depth=5, learning_rate=0.1, random_state=42)
elif 'Random Forest' in best_model_name:
    final_model = RandomForestRegressor(n_estimators=100, max_depth=10, random_state=42, n_jobs=-1)
elif 'Gradient Boosting' in best_model_name:
    final_model = GradientBoostingRegressor(n_estimators=100, max_depth=5, random_state=42)
elif 'Ridge' in best_model_name:
    final_model = Ridge(alpha=1.0)
elif 'Lasso' in best_model_name:
    final_model = Lasso(alpha=1.0)
elif 'Prophet' in best_model_name:
    final_model = Prophet(
        yearly_seasonality=True, 
        weekly_seasonality=False, 
        daily_seasonality=False,
        seasonality_mode='multiplicative',
        changepoint_prior_scale=0.05
    )
    final_model.add_seasonality(name='quarterly', period=91.25, fourier_order=4)
    prophet_full = df_train_full[[DATE_COLUMN, TARGET_COLUMN]].rename(columns={DATE_COLUMN: 'ds', TARGET_COLUMN: 'y'})
    final_model.fit(prophet_full)
else:
    final_model = LinearRegression()

if 'Prophet' not in best_model_name:
    final_model.fit(X_full, y_full)
    
print("Retraining complete!")

In [ ]:
# Make future predictions
if len(df_predict) > 0:
    print("\nMaking future predictions...")
    
    if 'Prophet' in best_model_name:
        future_prophet = df_predict[[DATE_COLUMN]].rename(columns={DATE_COLUMN: 'ds'})
        forecast = final_model.predict(future_prophet)
        future_predictions = forecast['yhat'].values
    else:
        X_future = df_predict[feature_cols_extended]
        future_predictions = final_model.predict(X_future)
    
    # Add to dataframe
    df_predict[f'{TARGET_COLUMN}_prediction'] = future_predictions
    
    print("\nFuture prediction summary (monthly):")
    summary = df_predict[[DATE_COLUMN, f'{TARGET_COLUMN}_prediction']].copy()
    summary['Year-Month'] = summary[DATE_COLUMN].dt.strftime('%Y-%m')
    summary[f'{TARGET_COLUMN}_prediction ({TARGET_UNIT})'] = (summary[f'{TARGET_COLUMN}_prediction'] / TARGET_SCALE).round(2)
    print(summary[['Year-Month', f'{TARGET_COLUMN}_prediction ({TARGET_UNIT})']].to_string(index=False))
    
    print(f"\nTotal prediction for forecast period: {df_predict[f'{TARGET_COLUMN}_prediction'].sum() / TARGET_SCALE:,.2f} {TARGET_UNIT}")
else:
    print("\nNo future data to predict")

In [ ]:
# Visualize future predictions
if len(df_predict) > 0:
    fig, ax = plt.subplots(figsize=(14, 6))
    
    # Historical actual values
    ax.plot(df_train_full[DATE_COLUMN], df_train_full[TARGET_COLUMN] / TARGET_SCALE, 'b-', 
            label='Actual', linewidth=1.5, alpha=0.7, marker='o', markersize=3)
    
    # Future predictions
    ax.plot(df_predict[DATE_COLUMN], df_predict[f'{TARGET_COLUMN}_prediction'] / TARGET_SCALE, 'r--', 
            label='Prediction', linewidth=2, marker='s', markersize=5)
    
    # Boundary line
    ax.axvline(x=df_train_full[DATE_COLUMN].max(), color='gray', linestyle=':', alpha=0.7, linewidth=2)
    ax.text(df_train_full[DATE_COLUMN].max(), ax.get_ylim()[1]*0.95, ' Forecast Start', 
            verticalalignment='top', fontsize=10, color='gray')
    
    ax.set_title(f'{TARGET_DISPLAY_NAME} Forecast (Actual + Future Prediction)', fontsize=14)
    ax.set_xlabel('Date')
    ax.set_ylabel(f'{TARGET_DISPLAY_NAME} ({TARGET_UNIT})')
    ax.legend()
    ax.grid(True, alpha=0.3)
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()
else:
    print("No future predictions to visualize")

## 13. Export Results

In [ ]:
# Export predictions to CSV
if len(df_predict) > 0:
    export_cols = [DATE_COLUMN, f'{TARGET_COLUMN}_prediction']
    if EXCLUDE_COLUMNS and EXCLUDE_COLUMNS[0] in df_predict.columns:
        export_cols.insert(1, EXCLUDE_COLUMNS[0])
    
    export_df = df_predict[export_cols].copy()
    export_df[DATE_COLUMN] = export_df[DATE_COLUMN].dt.strftime('%Y-%m')
    export_df[f'{TARGET_COLUMN}_prediction'] = export_df[f'{TARGET_COLUMN}_prediction'].round(2)
    export_df.to_csv('forecast_results_monthly.csv', index=False, encoding='utf-8-sig')
    print("Predictions saved to 'forecast_results_monthly.csv'")
    
    # Download
    files.download('forecast_results_monthly.csv')
else:
    print("No predictions to export")

## 14. Summary

In [ ]:
print("\n" + "="*80)
print("ANALYSIS SUMMARY (MONTHLY TIME SERIES)")
print("="*80)

print("\n[Data Overview]")
print(f"  Training period: {df_train_full[DATE_COLUMN].min().strftime('%Y-%m')} to {df_train_full[DATE_COLUMN].max().strftime('%Y-%m')}")
if len(df_predict) > 0:
    print(f"  Forecast period: {df_predict[DATE_COLUMN].min().strftime('%Y-%m')} to {df_predict[DATE_COLUMN].max().strftime('%Y-%m')}")
print(f"  Target variable: {TARGET_COLUMN}")
print(f"  Number of features: {len(feature_cols_extended)}")

print("\n[Model Comparison Results (by MAPE)]")
print(results_display.to_string(index=False))

print(f"\n[Best Model]")
print(f"  {best_model_name}")
print(f"  MAPE: {best_mape:.2f}%")

print("\n[Key Insights for Monthly Data]")
print("""
- Monthly time series features focus on yearly and quarterly seasonality
- Cyclical features (sin/cos) capture 12-month and 4-quarter patterns
- Year start/end and quarter boundaries often show special patterns
- Tree-based models (LightGBM, XGBoost) excel at non-linear relationships
- Prophet with custom quarterly seasonality works well for monthly data
- Feature importance helps identify key drivers (trends vs seasonality)
""")